In [32]:
import pandas as pd

import os

import numpy as np

from matplotlib import pyplot
import matplotlib as mpl

from scipy import optimize

from scipy.io import loadmat

import utils

grader = utils.Grader()

%matplotlib inline

In [2]:
df1 = pd.read_csv('Hotel_details.csv')
df1.head()

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [3]:
df1.isnull().sum()

id                  0
hotelid             0
hotelname           0
address          5093
city                0
country             0
zipcode         24562
propertytype        0
starrating          0
latitude            0
longitude           0
Source              0
url               111
curr                0
dtype: int64

In [4]:
df1 = df1.drop(['address', 'zipcode', 'url'], axis=1)

In [5]:
df1.head()

,id,hotelid,hotelname,city,country,propertytype,starrating,latitude,longitude,Source,curr
0,46406,1771651,Mediteran Bungalow Galeb,Omis,Croatia,Holiday parks,4,43.440124,16.682505,2,EUR
1,46407,177167,Hotel Polonia,Torun,Poland,Hotels,3,53.012329,18.603800,5,EUR
2,46408,1771675,Rifugio Sass Bece,Canazei,Italy,Hotels,3,46.477920,11.813350,2,EUR
3,46409,177168,Madalena Hotel,Mykonos,Greece,Hotels,3,37.452316,25.329849,5,EUR
4,46410,1771718,Pension Morenfeld,Lagundo,Italy,Hotels,3,46.682780,11.131736,2,EUR


In [6]:
df1 = df1.rename(columns={'id': 'custid'})

In [7]:
df1.head()

,custid,hotelid,hotelname,city,country,propertytype,starrating,latitude,longitude,Source,curr
0,46406,1771651,Mediteran Bungalow Galeb,Omis,Croatia,Holiday parks,4,43.440124,16.682505,2,EUR
1,46407,177167,Hotel Polonia,Torun,Poland,Hotels,3,53.012329,18.603800,5,EUR
2,46408,1771675,Rifugio Sass Bece,Canazei,Italy,Hotels,3,46.477920,11.813350,2,EUR
3,46409,177168,Madalena Hotel,Mykonos,Greece,Hotels,3,37.452316,25.329849,5,EUR
4,46410,1771718,Pension Morenfeld,Lagundo,Italy,Hotels,3,46.682780,11.131736,2,EUR


In [8]:
dt = df1[['custid', 'hotelid', 'hotelname', 'starrating']]
dt.head()

,custid,hotelid,hotelname,starrating
0,46406,1771651,Mediteran Bungalow Galeb,4
1,46407,177167,Hotel Polonia,3
2,46408,1771675,Rifugio Sass Bece,3
3,46409,177168,Madalena Hotel,3
4,46410,1771718,Pension Morenfeld,3


In [39]:
dt.starrating.value_counts()

3    56419
4    35892
2    15737
Name: starrating, dtype: int64

In [9]:
dt = dt.drop_duplicates(['custid', 'hotelname'])

In [10]:
hotel_count = dt.groupby('hotelname').agg('count')
hotel_count

,custid,hotelid,starrating
hotelname,,,
Eurostars Monumento Monasterio de San Clodio Hotel & Spa,1,1,1
Exe Las Margas Golf,1,1,1
Hotel La Selva,1,1,1
""" The One '"", Adults Only Boutique & Luxury Design Hotel",1,1,1
"""Het Wapen Van Willemstad'""",1,1,1
...,...,...,...
whala!beach,1,1,1
zeitlos Hotel und Restaurant am Fuchsbach,1,1,1
zeitwohnhaus SUITE-HOTEL & SERVICED APARTMENTS,1,1,1


In [11]:
hotel_count = hotel_count.reset_index().iloc[:,:2]
hotel_count = hotel_count.rename(columns={'custid':'hotelcount'})
hotel_count 

,hotelname,hotelcount
0,Eurostars Monumento Monasterio de San Clodio ...,1
1,Exe Las Margas Golf,1
2,Hotel La Selva,1
3,""" The One '"", Adults Only Boutique & Luxury De...",1
4,"""Het Wapen Van Willemstad'""",1
...,...,...
96198,whala!beach,1
96199,zeitlos Hotel und Restaurant am Fuchsbach,1
96200,zeitwohnhaus SUITE-HOTEL & SERVICED APARTMENTS,1
96201,zenit,1


In [12]:
dff = pd.merge(dt, hotel_count, how='left', on='hotelname')
dff.head()

,custid,hotelid,hotelname,starrating,hotelcount
0,46406,1771651,Mediteran Bungalow Galeb,4,1
1,46407,177167,Hotel Polonia,3,8
2,46408,1771675,Rifugio Sass Bece,3,1
3,46409,177168,Madalena Hotel,3,1
4,46410,1771718,Pension Morenfeld,3,1


In [13]:
dff.shape

(108048, 5)

In [14]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108048 entries, 0 to 108047
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   custid      108048 non-null  int64 
 1   hotelid     108048 non-null  int64 
 2   hotelname   108048 non-null  object
 3   starrating  108048 non-null  int64 
 4   hotelcount  108048 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 4.9+ MB


In [15]:
import numpy as np

In [16]:
dff['hotelcount'].quantile(np.arange(0.5, 1, 0.05))

0.50    1.0
0.55    1.0
0.60    1.0
0.65    1.0
0.70    1.0
0.75    1.0
0.80    1.0
0.85    2.0
0.90    3.0
0.95    7.0
Name: hotelcount, dtype: float64

In [17]:
dff.head()

,custid,hotelid,hotelname,starrating,hotelcount
0,46406,1771651,Mediteran Bungalow Galeb,4,1
1,46407,177167,Hotel Polonia,3,8
2,46408,1771675,Rifugio Sass Bece,3,1
3,46409,177168,Madalena Hotel,3,1
4,46410,1771718,Pension Morenfeld,3,1


In [18]:
dff_final = dff[dff['hotelcount']>1]
dff_final_pivot = dff_final.pivot(index='hotelname', columns='custid', values='starrating')
#dff_final_pivot = book_rating_pivot.fillna(0)
dff_final_pivot.head()

custid,46407,46415,46418,46436,46441,46442,46449,46458,46461,46462,...,154350,154360,154367,154379,154386,154389,154394,154407,154425,154437
hotelname,,,,,,,,,,,,,,,,,,,,,
A3 Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABC Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ART Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AS Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abacus Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
dff_final_pivot = dff_final_pivot.fillna(0)
dff_final_pivot.head()

custid,46407,46415,46418,46436,46441,46442,46449,46458,46461,46462,...,154350,154360,154367,154379,154386,154389,154394,154407,154425,154437
hotelname,,,,,,,,,,,,,,,,,,,,,
A3 Hotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABC Hotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ART Hotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AS Hotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abacus Hotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
dff_final_pivot.shape

(4405, 16250)

In [21]:
Y = dff_final_pivot.values.reshape(4405, 16250)
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
Y[Y>0]

array([3., 2., 3., ..., 3., 3., 3.])

In [23]:
Y.shape

(4405, 16250)

In [25]:
R = []
for j in range(Y.shape[1]):
    
    for ele in Y:
        if ele[j]==0:
            R.append(0)
        else:
            R.append(1)
            
R = np.array(R)

In [26]:
R = R.reshape(4405,16250,  order='F')

In [27]:
R

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [28]:
R.shape

(4405, 16250)

In [30]:
hotel_title = dff_final_pivot.index
hotel_list = list(hotel_title)
hotel_list

['A3 Hotel',
 'ABC Hotel',
 'ART Hotel',
 'AS Hotel',
 'Abacus Hotel',
 'Abbey Hotel',
 'Abc Hotel',
 'Academy Hotel',
 'Acapulco Hotel',
 'Achillion Hotel',
 'Acorn Lodge',
 'Acropol Hotel',
 'Acropolis Hotel',
 'Actor Hotel',
 'Adams Hotel',
 'Adler Hotel',
 'Admiral Hotel',
 'Adonis Hotel',
 'Aegeon Hotel',
 'Aegli Hotel',
 'Aeolis Hotel',
 'Aeolos Hotel',
 'Aeroport Hotel',
 'Afrodite Hotel',
 'Agali Hotel',
 'Agate Hotel',
 'Agave Hotel',
 'Agnadi Hotel',
 'Agnanti Hotel',
 'Ahotel',
 'Air Hotel',
 'Airone Hotel',
 'Aisling Bed & Breakfast',
 'Akrogiali Hotel',
 'Akrotiri Hotel',
 'Aktaion',
 'Aktaion Hotel',
 'Akti',
 'Akti Hotel',
 'Aktivhotel Crystal',
 'Al Caminetto',
 'Al Castello',
 'Al Cavallino Bianco',
 'Al Giardinetto',
 'Al Piccolo Hotel',
 'Alameda',
 'Alba Hotel',
 'Albatros',
 'Albatros Hotel',
 'Albergo Al Castello',
 'Albergo Al Parco',
 'Albergo Al Sole',
 'Albergo Alla Campana',
 'Albergo Alla Rosa',
 'Albergo Alpino',
 'Albergo Altavilla',
 'Albergo Antica Locan

In [29]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):

    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)


    
    J = (1/2)*np.sum((np.square(np.dot(X, Theta.T) - Y)*R)) + (lambda_/2)*np.sum(np.square(X)) + (lambda_/2)*np.sum(np.square(Theta))
    
    for i in range(R.shape[0]):
        
        idx = np.where(R[i, :] == 1)[0]
        Theta_temp = Theta[idx, :]
        Y_temp = Y[i, idx]
        X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]
    
    for j in range(R.shape[1]):
        
        idx = np.where(R[:, j] == 1)[0]
        X_temp = X[idx, :]
        Y_temp = Y[idx, j]
        Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]
            
  
    
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return J, grad

In [33]:

n_m = len(hotel_list)


my_ratings = np.zeros(n_m)

Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

Ynorm, Ymean = utils.normalizeRatings(Y, R)

num_movies, num_users = Y.shape
num_features = 10

X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])


options = {'maxiter': 100}


lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

print('Recommender system learning completed.')

Recommender system learning completed.


In [45]:
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

#movieList = utils.loadMovieList()
hotel_list = list(hotel_title)

ix = np.argsort(my_predictions)[::-1]

print('Top recommendations for Cutomers:')
print('----------------------------')
for i in range(500):
    j = ix[i]
    print('Predicting rating %.4f for hotel %s' % (my_predictions[j], hotel_list[j]))

print('\nOriginal ratings provided:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d for %s' % (my_ratings[i], hotel_list[i]))

Top recommendations for Cutomers:
----------------------------
Predicting rating 4.0000 for hotel The Rutland Hotel
Predicting rating 4.0000 for hotel Mill Farm
Predicting rating 4.0000 for hotel Gwynfryn
Predicting rating 4.0000 for hotel Hotel Delle Palme
Predicting rating 4.0000 for hotel Mill House
Predicting rating 4.0000 for hotel Oakwood
Predicting rating 4.0000 for hotel The Pheasant Inn
Predicting rating 4.0000 for hotel Hotel Anda
Predicting rating 4.0000 for hotel Grand Hotel Capodimonte
Predicting rating 4.0000 for hotel Grand Beach Hotel
Predicting rating 4.0000 for hotel Cavalieri Hotel
Predicting rating 4.0000 for hotel Duca D'Aosta Hotel
Predicting rating 4.0000 for hotel Blue Horizon
Predicting rating 4.0000 for hotel Pilio Holiday Club
Predicting rating 4.0000 for hotel Y Bwthyn
Predicting rating 4.0000 for hotel Sea View
Predicting rating 4.0000 for hotel The Old Stables
Predicting rating 4.0000 for hotel Granary Cottage
Predicting rating 4.0000 for hotel Hotel Crocu

In [35]:
my_ratings

array([0., 0., 0., ..., 0., 0., 0.])